# 以相似度为训练目标

In [0]:
from google.colab import drive
drive.mount('/content/drive')

## 训练数据生成器

In [0]:
!git clone https://github.com/daibiaoxuwu/05wan8
!git clone https://github.com/daibiaoxuwu/05wan8b

In [0]:
import os
import cv2
import numpy as np
import random
from matplotlib import pyplot as plt
def read_data(path1,path2):

  x_train0 = []
  y_train0 = dict()
  idxes = dict()

  #read clock
  #read and /255
  for filename in os.listdir(path2):
      if 'git' in filename: continue
      y_train0[int(filename[:-4])]=cv2.imread(os.path.join(path2,filename), cv2.IMREAD_GRAYSCALE).astype('float32') /255.0      

  keys = list(y_train0.keys())
  
  #read data
  #add an axis and /255
  
  filelist = os.listdir(path1)
  random.shuffle(filelist)
  index = 0
  for filename in filelist:
      if 'git' in filename or 'c.jpg' in filename: continue
      ftime = int(filename.split('_')[0])
      value = max(filter(lambda t:t<ftime,keys))
      if(abs(ftime-value)<10000):continue

      x_train0.append(cv2.imread(os.path.join(path1,filename), cv2.IMREAD_GRAYSCALE).astype('float32')[:,:,np.newaxis]/255.0)
      if value not in idxes:idxes[value]=[index]
      else: idxes[value].append(index)
      index += 1

  #index data
  print(len(x_train0),index,len(y_train0),len(idxes.keys()))
  

  #generator
  def data_generator(ikeys_t):
    while True:
      dataout = []
      for i in range(5):dataout.append([])
      ansout = []
      for train2 in range(100):
        key = random.choice(ikeys_t)
        idx2 = random.sample(idxes[key],5)
        for i in range(5):
          dataout[i].append(x_train0[idx2[i]])
        ansout.append(y_train0[key])
      yield [np.stack(data2) for data2 in dataout],np.stack(ansout)
  
  #split
  ikeys = list(idxes.keys())
  random.shuffle(ikeys)
  ltrain = int(len(ikeys)*0.8)  
  train_generator = data_generator(ikeys[:ltrain])
  test_generator = data_generator(ikeys[ltrain:])

  return train_generator,test_generator



In [0]:
train_generator,test_generator = read_data('05wan8','05wan8b')

# 数据初始化完成。


# 装载Tensorflow

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# 安装 TensorFlow
try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf


# 搭建模型和训练
现在的网络去掉dropout则可以拟合10张图片，
但是加上dropout就拟合不了。
从7张拟合5张的，则可以在两个epoch（每个epoch抽取60000次）内到80%。

继续训练20张，一个epoch就到60%。

In [0]:
picnum = 5
inputs = [tf.keras.layers.Input(shape=(21,45,1))  for i in range(picnum)]
#inputs = tf.keras.layers.Input(shape=(5,12,12,1))

conv_a1 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')
conva1 = [conv_a1(inputs[i]) for i in range(picnum)]
conv_b1 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')
convb1 = [conv_b1(conva1[i]) for i in range(picnum)]

conv_c1 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')
convc1 = [conv_c1(inputs[i]) for i in range(picnum)]
conv_d1 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')
convd1 = [conv_d1(convc1[i]) for i in range(picnum)]

convs1 = tf.stack(convb1)
convf1 = [
    tf.math.reduce_max(convs1, axis=[0]),
    tf.math.reduce_mean(convs1, axis=[0]),
    tf.math.reduce_min(convs1, axis=[0]),
]

conv_g1 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')
convg1 = [conv_g1(convf1[i]) for i in range(3)]

convh1 = [tf.concat(convg1+[convd1[i]],axis=3) for i in range(picnum)]
#------------------------------------------------------------------------------------------
conv_a2 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')
conva2 = [conv_a2(convh1[i]) for i in range(picnum)]
conv_b2 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')
convb2 = [conv_b2(conva2[i]) for i in range(picnum)]

conv_c2 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')
convc2 = [conv_c2(inputs[i]) for i in range(picnum)]
conv_d2 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')
convd2 = [conv_d2(convc2[i]) for i in range(picnum)]

convs2 = tf.stack(convb2)
convf2 = [
    tf.math.reduce_max(convs2, axis=[0]),
    tf.math.reduce_mean(convs2, axis=[0]),
    tf.math.reduce_min(convs2, axis=[0]),
]

conv_g2 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')
convg2 = [conv_g2(convf2[i]) for i in range(3)]

convh2 = [tf.concat(convg2+[convd2[i]],axis=3) for i in range(picnum)]

#------------------------------------------------------------------------------------------
conv_a3 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')
conva3 = [conv_a3(convh2[i]) for i in range(picnum)]
conv_b3 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')
convb3 = [conv_b3(conva3[i]) for i in range(picnum)]

conv_c3 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')
convc3 = [conv_c3(inputs[i]) for i in range(picnum)]
conv_d3 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')
convd3 = [conv_d3(convc3[i]) for i in range(picnum)]

convs3 = tf.stack(convb3)
convf3 = [
    tf.math.reduce_max(convs3, axis=[0]),
    tf.math.reduce_mean(convs3, axis=[0]),
    tf.math.reduce_min(convs3, axis=[0]),
]

conv_g3 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')
convg3 = [conv_g3(convf3[i]) for i in range(3)]

convh3 = tf.concat(convg3+convd3,axis=3)

convx1 = tf.keras.layers.Conv2D(256, (3, 3), padding = 'Same',  activation='relu')(convh3)
convx3 = tf.keras.layers.Conv2D(64, (3, 3), padding = 'Same', activation='relu')(convx1)
'''
convx3 = tf.keras.layers.Flatten()(convx2)

#keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)
convx4 = tf.keras.layers.Dropout(0.2)(convx3)

convy1 = tf.keras.layers.Dense(21*45)(convx4)
#convy3 = tf.keras.layers.Dense(10,activation='softmax')(convy1)
convy4 = tf.keras.layers.Reshape([21,45])(convy1)
'''

convx4 = tf.keras.layers.Conv2D(1, (1, 1), padding = 'Same',  activation='relu')(convx3)

model = tf.keras.Model(inputs=inputs, outputs=convx4)


In [0]:
#tf.reduce_mean(tf.image.ssim(reconstructed, truth, 1.0))
model.compile(optimizer='adam',#tf.keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
                #loss=lambda y_pred,y_true:1-tf.reduce_mean(tf.image.ssim_multiscale(tf.clip_by_value(y_pred,0,1), y_true, 255))#-tf.reduce_mean(tf.image.psnr(tf.clip_by_value(y_pred,0,1), y_true, 1.0))
              loss='mse',
              metrics=['mse']
              )
            
#model.summary()

# 开始训练

In [0]:
'''
import math
#reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=5, mode='auto')
def step_decay(epoch):
  initial_lrate = 0.001
  drop = 0.5
  epochs_drop = 10.0
  lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
  return lrate
  
reduce_lr = tf.keras.callbacks.LearningRateScheduler(step_decay)
'''
#model.load_weights('/content/drive/My Drive/checkpoints/checkpoints/my_checkpoint')
#os.mkdir('/content/drive/My Drive/checkpoints')
checkpoint_path = "/content/drive/My Drive/checkpoints/training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
os.mkdir(checkpoint_dir)
#model.load_weights(checkpoint_path)
# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=False,
                                                 verbose=0)


model.fit_generator(train_generator,epochs=200,steps_per_epoch=2000,validation_data=test_generator,validation_steps=50,validation_freq=1,callbacks=[cp_callback])#,callbacks=[reduce_lr])

# 测试正确率

In [0]:
data,ans = next(test_generator)
data = [data[x][5][np.newaxis,:,:,:] for x in range(5)]
ans = ans[5]
import matplotlib
cmap = matplotlib.cm.gray 
cmap.set_bad(color='black')

pred = model.predict(data)
pred[0,0,0,0]=0
ans[0,0]=0
for i in range(5):
  plt.subplot(321+i)
  plt.imshow(data[i][0,:,:,0],cmap=cmap)
plt.show()
plt.imshow(np.clip(pred[0,:,:,0],0,1),cmap=cmap)
print(pred[0,0,0])

plt.show()
plt.imshow(pred[0,:,:,0],cmap=cmap)
plt.show()
plt.imshow(ans,cmap=cmap)
plt.show()
from skimage.metrics import peak_signal_noise_ratio,structural_similarity, mean_squared_error
print(peak_signal_noise_ratio(np.clip(pred[0,:,:,0],0,1),ans))
print(structural_similarity(np.clip(pred[0,:,:,0],0,1),ans))
print(mean_squared_error(np.clip(pred[0,:,:,0],0,1),ans))
pic = np.ones(ans.shape)
print(mean_squared_error(np.clip(pic,0,1),ans))
#ssims = tf.image.ssim(tf.clip_by_value(pred[0],0,1), ans, 1.0)


In [0]:
from skimage.metrics import mean_squared_error as mse
from skimage.metrics import peak_signal_noise_ratio,structural_similarity, mean_squared_error

bigdiff = 0
bigdata = 0
bigans = 0
bigpred = 0
for it in range(1):
  data,ans = next(train_generator)
  pred = model.predict(data)
  for it2 in range(100):
    data2 = [data[x][it2][np.newaxis,:,:,:] for x in range(5)]
    ans2 = ans[it2]
    
    diff = structural_similarity(np.clip(pred[it2,:,:,0],0,1),ans2)
    if(diff > bigdiff):
      bigdiff=diff
      bigdata=data2
      bigans=ans2
      bigpred=pred[it2,:,:,0]


for i in range(5):
  plt.subplot(321+i)
  plt.imshow(bigdata[i][0,:,:,0],cmap='gray')
plt.show()
plt.imshow(bigpred,cmap='gray')
plt.show()
plt.imshow(bigans,cmap='gray')
plt.show()
print(bigdiff)
print(mean_squared_error(np.clip(bigpred,0,1),bigans))



 

In [0]:
from skimage.metrics import mean_squared_error as mse
from skimage.metrics import peak_signal_noise_ratio,structural_similarity

bigdiff = 0
bigdata = 0
bigans = 0
bigpred = 0
diffs=[]
for it in range(100):
  data,ans = next(train_generator)
  pred = model2.predict(data)
  diffs2 = []
  for it2 in range(100):
    data2 = [data[x][it2][np.newaxis,:,:,:] for x in range(5)]
    ans2 = ans[it2]
     #   print(ans2.dtype)
    #print(pred.dtype)
#    print(ans2.max(),ans2.min())
#    print(pred[it2,:,:,0].max(),pred[it2,:,:,0].min())
    diff = structural_similarity(np.clip(pred[it2,:,:,0],0,1),ans2)
    diffs2.append(diff)
  diffs.append(np.average(diffs2))
plt.hist(diffs)